# Contexto
- Utilizando la playlist de "Canciones que te gustan" obtenemos todos los tracks de ahí
- También obtendremos todos los "Top tracks" por si el usuario no tuviera de lo otro
- La idea es sacar lo mismo que en las marcas:
    - Top artistas
    - Top géneros
    - Top subgéneros

In [7]:
import sys
sys.path.append("../")
import pandas as pd
import src.soporte_spotify as api
import src.soporte_sql as sql

In [8]:
sp = api.load_credentials()

In [ ]:
def get_current_user(sp):
    user_data = sp.current_user()
    user = pd.DataFrame({
        "user_id" : user_data["id"],
        "name" : user_data["display_name"],
        "url" : user_data["external_urls"]["spotify"],
        "email" : user_data["email"],
        "product_version" : user_data["product"]
    })
    conexion = sql.conectar_bd()
    query = '''INSERT INTO users(user_id,name,url,email,product_version) VALUES (%s,%s,%s,%s,%s)'''
    sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(user))

get_current_user(sp)

In [41]:
def get_all_saved_tracks(spotify_client, limit=50):

    all_tracks = []  # Lista para almacenar todas las canciones
    results = spotify_client.current_user_saved_tracks(limit=limit)  # Primera página

    while results:
        # Añadir las canciones de la página actual a la lista
        all_tracks.extend(results['items'])

        # Seguir al siguiente enlace, si existe
        if results['next']:
            results = spotify_client.next(results)
        else:
            break
    
    users_ids = []
    songs_names = []
    songs_ids = []
    popularities_list = []
    songs_urls = []
    artists_names = []
    artists_ids = []
    artists_urls = []
    added_at_list = []
    user_id = sp.current_user()["id"]
    for track in all_tracks:
        # User id
        users_ids.append(user_id)
        #Nombre Cancion
        songs_names.append(track["track"]["name"])
        #Id Cancion
        songs_ids.append(track["track"]["id"])
        # Popularidad
        popularities_list.append(track["track"]["popularity"])
        # Url Cancion
        songs_urls.append(track["track"]["external_urls"]["spotify"])
        # Nombre del Artista
        artists_names.append(track["track"]["artists"][0]["name"])
        # Id del Artista
        artists_ids.append(track["track"]["artists"][0]["id"])
        # Url de Spotify del Artista
        artists_urls.append(track["track"]["artists"][0]["external_urls"]["spotify"])
        # Fecha en la que el usuario la añadio a su lista
        added_at_list.append(track["added_at"].split("T")[0])

    user_liked_songs = pd.DataFrame({
    "user_id" : users_ids,
    "song_name" : songs_names,
    "song_id" : songs_ids,
    "popularity" : popularities_list,
    "song_url" : songs_urls,
    "artist_name" : artists_names,
    "artist_id" : artists_ids,
    "artist_url" : artists_urls,
    "user_added_at" : added_at_list
    })

    conexion = sql.conectar_bd()
    query = '''INSERT INTO users(user_id,song_name,song_id,popularity,song_url,artist_name,artist_id,artist_url,user_added_at) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
    sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(user_liked_songs))

get_all_saved_tracks(sp)

,user_id,song_name,song_id,popularity,song_url,artist_name,artist_id,artist_url,user_added_at
0,gonzaloruiperez,Can Can World,4aqu6TwPQMhL2kJyJp7la6,8,https://open.spotify.com/track/4aqu6TwPQMhL2kJ...,Makkeroni,3wHUW8a3eU0wZydKOWlFan,https://open.spotify.com/artist/3wHUW8a3eU0wZy...,2025-01-20
1,gonzaloruiperez,PRIME TURBO,6HOf5mZay69GSjqCcpsEp8,28,https://open.spotify.com/track/6HOf5mZay69GSjq...,Jameskii,4Hsmk6llKgAiIj2GrxrQsl,https://open.spotify.com/artist/4Hsmk6llKgAiIj...,2025-01-16
2,gonzaloruiperez,DUAL ROZES (feat. Nana Takahashi/DJ Command(Eu...,1uhccpbBahDIdTy6wjKlMU,38,https://open.spotify.com/track/1uhccpbBahDIdTy...,SOUND HOLIC,3zzEf10VyiuuFYPXkmfn9i,https://open.spotify.com/artist/3zzEf10VyiuuFY...,2025-01-16
3,gonzaloruiperez,Atlantis,1A67uZiZxZbhuhHbS1THye,49,https://open.spotify.com/track/1A67uZiZxZbhuhH...,Netherworld,79LaPRYSRh7JAnu3r1jYPF,https://open.spotify.com/artist/79LaPRYSRh7JAn...,2024-12-30
4,gonzaloruiperez,Break My Stride,1mCsF9Tw4AkIZOjvZbZZdT,69,https://open.spotify.com/track/1mCsF9Tw4AkIZOj...,Matthew Wilder,3bmFPbLMiLxtR9tFrTcKcP,https://open.spotify.com/artist/3bmFPbLMiLxtR9...,2024-11-29
...,...,...,...,...,...,...,...,...,...
168,gonzaloruiperez,Todo De Ti,4fSIb4hdOQ151TILNsSEaF,71,https://open.spotify.com/track/4fSIb4hdOQ151TI...,Rauw Alejandro,1mcTU81TzQhprhouKaTkpq,https://open.spotify.com/artist/1mcTU81TzQhprh...,2021-07-17
169,gonzaloruiperez,Call Me a Spaceman (feat. Mitch Crown) - Radio...,40wyRyWjLI00OE9gxh9WbL,0,https://open.spotify.com/track/40wyRyWjLI00OE9...,Hardwell,6BrvowZBreEkXzJQMpL174,https://open.spotify.com/artist/6BrvowZBreEkXz...,2015-07-09
170,gonzaloruiperez,Spaceman - Headhunterz Remix,20n9rTm3Vyuj3ptQgTbfxF,0,https://open.spotify.com/track/20n9rTm3Vyuj3pt...,Hardwell,6BrvowZBreEkXzJQMpL174,https://open.spotify.com/artist/6BrvowZBreEkXz...,2015-07-09
171,gonzaloruiperez,Spaceman - Drown The Fish Remix,2mwX65kNGJGzAUk9Ik4LkN,0,https://open.spotify.com/track/2mwX65kNGJGzAUk...,Hardwell,6BrvowZBreEkXzJQMpL174,https://open.spotify.com/artist/6BrvowZBreEkXz...,2015-07-09
